# Roteamento de cidades com Busca A*

Na busca A* é usado a heurística h(n) (custo até o objetivo) em conjunto com a heurística grafo(n) (custo de cada nó) formando:

*f(n) = h(n) + grafo(n)*

### Modelando Nó e Grafo:

In [ ]:
class Grafo:

  #O construtor cria um grafo, permitindo que ele seja direcionado (padrão) ou não direcionado.
  #Um grafo direcionado permite conexões unidirecionais entre nós, enquanto um não direcionado permite conexões bidirecionais.
  def __init__(self, dict_grafo=None, direcionado=True):
    self.dict_grafo = dict_grafo or {}
    self.direcionado = direcionado
    if not direcionado:
      self.tornar_nao_direcionado()

  #Este método converte um grafo direcionado em um grafo não direcionado.
  #Ele faz isso adicionando arestas reversas para cada aresta existente no grafo. Isso garante que todas as conexões sejam bidirecionais.
  def tornar_nao_direcionado(self):
    for a in list(self.dict_grafo.keys()):
      for (b, dist) in self.dict_grafo[a].items():
        self.dict_grafo.setdefault(b, {})[a] = dist

  #Adicionar uma conexão entre dois nós, A e B, no grafo. Você pode fornecer
  #uma distância opcional (o padrão é 1) para a conexão. Se o grafo for não direcionado, ele também adicionará a conexão na direção oposta.
  def conectar(self, A, B, distancia=1):
    self.dict_grafo.setdefault(A, {})[B] = distancia
    if not self.direcionado:
      self.dict_grafo.setdefault(B, {})[A] = distancia

  #O método get permite obter informações sobre as conexões em um grafo.
  #Se apenas um argumento, a, for fornecido, ele retorna um dicionário das conexões do nó a.
  #Cada chave no dicionário é um nó conectado a a, e o valor é a distância entre a e esse nó.
  #Se dois argumentos, a e b, forem fornecidos, ele retorna a distância entre os nós a e b,
  #se houver uma conexão direta. Se não houver conexão, retorna None.
  def get(self, a, b=None):
    links = self.dict_grafo.setdefault(a, {})
    if b is None:
      return links
    else:
      return links.get(b)

  #Este método retorna uma lista de todos os nós no grafo. Ele faz isso combinando os nós
  #presentes nas chaves do dicionário dict_grafo e os nós presentes nas chaves dos dicionários aninhados no dicionário dict_grafo.
  def nos(self):
    s1 = set([k for k in self.dict_grafo.keys()])
    s2 = set([k2 for v in self.dict_grafo.values() for k2, v2 in v.items()])
    nos = s1.union(s2)
    return list(nos)

class No:

  #O construtor da classe No inicializa um nó com um nome e um nó pai. O parâmetro name é o nome do nó, e o parâmetro parent é o nome do nó pai
  #(usado para rastrear o caminho durante a busca). Além disso, ele inicializa a distância acumulada g como 0 e a distância heurística h como 0.
  def __init__(self, name:str, parent:str):
    self.name = name
    self.parent = parent
    self.g = 0 # distancia para o nó inicial
    self.h = 0 # distancia para o nó final

  #Este método calcula e retorna o valor de avaliação f para o nó. O valor f é a soma da distância acumulada g e da distância heurística h.
  #É usado para determinar a ordem de prioridade na fila de prioridade durante a busca A*.
  def get_f(self):
    return self.g + self.h

  #Este método permite comparar se dois nós são iguais com base em seus nomes. É usado para verificar se dois nós são idênticos.
  def __eq__(self, other):
    return self.name == other.name

  #Este método permite comparar a ordem de prioridade de dois nós com base em seus valores f.
  #É usado para determinar qual nó deve ser escolhido a seguir durante a busca A*.
  def __lt__(self, other):
    return self.f < other.f

  #Este método retorna uma representação legível do nó na forma "(nome, f)". É útil para depuração e exibição de resultados.
  def __repr__(self):
    return ('({0},{1})'.format(self.name, self.get_f()))

  #Este método calcula um valor de hash para o nó com base em seu nome. É usado para armazenar nós em estruturas de dados como conjuntos e dicionários.
  def __hash__(self):
    return hash(self.name)

### Implementando a Busca A*:

In [ ]:
import heapq

def obter_caminho(no_atual, inicial):
  '''
    Função para gerar o caminho percorrido entre dois estados
    Args:
        - no_atual: estado no_atual
        - inicial: estado inicial
    Return:
        - Caminho percorrido entre o estado inicial e final
  '''

  #Lista vazia chamada caminho que será usada para armazenar o caminho percorrido
  caminho = []


  #O loop while é usado para repetir a seguinte ação até que no_atual seja igual a inicial.
  #Isso significa que o loop continuará até que o caminho tenha retrocedido o suficiente para chegar ao estado inicial.
  #A cada iteração do loop, o código executa as seguintes ações:
  # 1 - Adiciona o nome do no_atual seguido de ': ' e o valor de g (a distância acumulada) à lista caminho. Isso representa um nó no caminho percorrido.
  # 2 - Atualiza o no_atual para ser o pai do nó atual. Isso move o loop para o nó pai seguinte, retrocedendo no caminho.
  # 3 - O loop continua até que no_atual seja igual a inicial, o que significa que chegamos ao estado inicial e retrocedemos ao longo do caminho completo.
  while no_atual != inicial:
    caminho.append(no_atual.name + ': ' + str(no_atual.g))
    no_atual = no_atual.parent
  caminho.append(inicial.name + ': ' + str(inicial.g))

  #O caminho é invertido (caminho[::-1]) antes de ser retornado como resultado. Isso é feito
  #porque o caminho foi construído a partir do no_atual até o inicial,
  #e a ordem precisa ser invertida para representar o caminho completo do inicial ao no_atual.
  return caminho[::-1]

def a_estrela(grafo: Grafo, heuristicas, inicio, fim):
  '''
    Algoritmo a*, dado um grafo, um cojunto de heurísticas, um estado/nó inicial e um estado/nó final,
    busca o caminho ótimo e completo
    Args:
        - grafo: grafo contendo os estados da busca
        - heuristicas: heuristica de cada estado
        - inicio: estado inicial
        - fim: estado final
    Return:
        - Melhor caminho entre estados inicial e final, se a solução foi encontrada, quantidade de estados visitados
  '''

  # Criando nó inicial e nó objetivo (não possuem pai)
  no_inicial = No(inicio, None)
  no_objetivo = No(fim, None)

  # Inicializando os atributos grafo(0) e h(0)
  no_inicial.grafo = 0
  no_inicial.h = heuristicas[inicio]

  id = 0

  # Fila de prioridades
  #Inicializa uma lista vazia chamada fila, que será usada como uma fila de prioridade.
  #O no_inicial é inserido na fila de prioridade com uma tupla que contém o valor de avaliação f do nó inicial, o id e o próprio no_inicial.
  fila = []
  #OBS:get_f() que calcula e retorna o valor de avaliação f do nó. O valor f geralmente é a soma da distância acumulada g e da distância heurística h.
  heapq.heappush(fila, (no_inicial.get_f(), id, no_inicial))

  # Nós criados
  nos_criados = []
  nos_criados.append(no_inicial)

  # Nós visitados
  nos_visitados = []

  # Controle
  solucao_encontrada = False
  caminho = None

  # Loop enquanto a fila não estiver vazia.
  while len(fila) != 0:
    # Pega na fila proxímo nó.
    no_atual = heapq.heappop(fila)[2]
    nos_visitados.append(no_atual)

    # Checar se já chegou na solução.
    if no_atual == no_objetivo:
      caminho = obter_caminho(no_atual, no_inicial)
      solucao_encontrada = True
      break

    # Percorrendo todos os vizinhos do nó atual.
    for nome, g in grafo.get(no_atual.name).items():
      # Criando nó com o nome do vizinho e passando o nó atual como pai.
      no_vizinho = No(nome, no_atual)

      # Checar se o nó vizinho já não foi criado.
      if no_vizinho not in nos_criados:
        # Atribuir valor de h(n) e g(n) ao nó.
        no_vizinho.g = no_atual.g + g
        no_vizinho.h = heuristicas[no_vizinho.name]

        # Inserir nó na lista de criados.
        nos_criados.append(no_vizinho)

        # Inserir nó na fila.
        id += 1
        heapq.heappush(fila, (no_vizinho.get_f(), id, no_vizinho))

  return solucao_encontrada, caminho, nos_visitados

### Modelando e executando problema:

![2](https://github.com/alvaromfcunha-c210/aula6/blob/master/imagens/2.png?raw=1)

In [ ]:
grafo = Grafo()

grafo.conectar('Santa Rita do Sapucaí', 'Pouso Alegre', 28.9)
grafo.conectar('Pouso Alegre', 'Cambui', 49.1)
grafo.conectar('Pouso Alegre', 'Congonhal', 24.3)
grafo.conectar('Pouso Alegre', 'Borda', 28.8)
grafo.conectar('Cambui', 'Camanducaia', 24.7)
grafo.conectar('Camanducaia', 'Braganca', 60.4)
grafo.conectar('Braganca', 'Atibaia', 25.2)
grafo.conectar('Braganca', 'Itapira', 82.4)
grafo.conectar('Atibaia', 'Campinas', 65.6)
grafo.conectar('Itapira', 'Campinas', 70.7)
grafo.conectar('Borda', 'Jacutinga', 57.6)
grafo.conectar('Jacutinga', 'Itapira', 33.2)
grafo.conectar('Congonhal', 'Ipuiuna', 24.6)
grafo.conectar('Ipuiuna', 'Andradas', 67.5)
grafo.conectar('Andradas', 'Espírito Santo do Pinhal', 28.4)
grafo.conectar('Espírito Santo do Pinhal', 'Mogi Guaçu', 35.7)
grafo.conectar('Mogi Guaçu', 'Mogi Mirim', 25)
grafo.conectar('Mogi Mirim', 'Campinas', 60.1)

grafo.tornar_nao_direcionado()

h = {}
h['Santa Rita do Sapucaí'] = 165
h['Pouso Alegre'] = 137
h['Cambui'] = 108
h['Camanducaia'] = 97
h['Braganca'] = 54
h['Atibaia'] = 57
h['Campinas'] = 0
h['Borda'] = 117
h['Jacutinga'] = 84
h['Itapira'] = 58
h['Congonhal'] = 135
h['Ipuiuna'] = 139
h['Andradas'] = 106
h['Espírito Santo do Pinhal'] = 86
h['Mogi Guaçu'] = 62
h['Mogi Mirim'] = 54

solucao_encontrada, caminho, nos_visitados = a_estrela(grafo, h, 'Santa Rita do Sapucaí', 'Campinas')

from pprint import pprint

print(f'Solução encontrada? {solucao_encontrada}')
print('Caminho encontrado:')
pprint(caminho)
print('Cidades (nós) visitados:')
pprint(nos_visitados)

Solução encontrada? True
Caminho encontrado:
['Santa Rita do Sapucaí: 0',
 'Pouso Alegre: 28.9',
 'Borda: 57.7',
 'Jacutinga: 115.30000000000001',
 'Itapira: 148.5',
 'Campinas: 219.2']
Cidades (nós) visitados:
[(Santa Rita do Sapucaí,165),
 (Pouso Alegre,165.9),
 (Borda,174.7),
 (Cambui,186.0),
 (Congonhal,188.2),
 (Jacutinga,199.3),
 (Camanducaia,199.7),
 (Itapira,206.5),
 (Ipuiuna,216.8),
 (Braganca,217.1),
 (Campinas,219.2)]


![3](https://github.com/alvaromfcunha-c210/aula6/blob/master/imagens/3.png?raw=1)